<a href="https://colab.research.google.com/github/aslinurtezcan/Laptop-Recommendation-Chat-Bot/blob/main/chatbot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder, StandardScaler
from sklearn.metrics.pairwise import cosine_similarity
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM
import numpy as np

# Veri setini yükleyelim
file_path = 'laptop_data.csv'  # Dosya yolunu doğru belirleyin
df = pd.read_csv(file_path)

# 'Weight' sütunundaki birimleri kaldırarak sadece sayısal değerleri alalım
df['Weight'] = df['Weight'].str.replace('kg', '').astype(float)

# 'Inches' ve 'Price' sütunlarını da float türüne çevirelim
df['Inches'] = df['Inches'].astype(float)
df['Price'] = df['Price'].astype(float)

# Kategorik verileri sayısal değerlere dönüştürelim
label_encoders = {}
for column in ['Company', 'TypeName', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys']:
    le = LabelEncoder()
    df[column] = le.fit_transform(df[column])
    label_encoders[column] = le

# Özellikler ve hedef değişken
X = df[['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']]
y = df[['Company', 'TypeName', 'Inches', 'ScreenResolution', 'Cpu', 'Ram', 'Memory', 'Gpu', 'OpSys', 'Weight', 'Price']]

# Veriyi eğitim ve test setlerine bölelim
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=35)

# Veriyi ölçeklendirelim
scaler = StandardScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)

# LSTM modelini tanımlayalım
model = Sequential()
model.add(LSTM(50, input_shape=(X_train.shape[1], 1)))
model.add(Dense(X_train.shape[1]))

model.compile(optimizer='adam', loss='mean_squared_error')

# Modeli eğitelim
X_train_lstm = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_lstm = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

history = model.fit(X_train_lstm, y_train, epochs=50, batch_size=32, validation_data=(X_test_lstm, y_test))

def get_user_preferences():
    preferences = {}
    preferences['Company'] = input("Lütfen istediğiniz bilgisayar markasını girin (örn. Apple, Dell): ")
    preferences['TypeName'] = input("Lütfen bilgisayar türünü girin (örn. Ultrabook, Gaming): ")
    preferences['Inches'] = float(input("Lütfen ekran boyutunu girin (örn. 13, 15.6): "))
    preferences['ScreenResolution'] = input("Lütfen ekran çözünürlüğünü girin (örn. 1920x1080): ")
    preferences['Cpu'] = input("Lütfen işlemci türünü girin (örn. Intel, AMD): ")
    preferences['Ram'] = input("Lütfen RAM miktarını girin (örn. 8GB, 16GB): ")
    preferences['Memory'] = input("Lütfen depolama türü ve miktarını girin (örn. 256GB SSD, 1TB HDD): ")
    preferences['Gpu'] = input("Lütfen grafik kartı türünü girin (örn. Nvidia, Intel): ")
    preferences['OpSys'] = input("Lütfen işletim sistemini girin (örn. Windows, macOS): ")
    preferences['Weight'] = float(input("Lütfen ağırlık sınırını girin (kg): "))
    preferences['Price'] = float(input("Lütfen fiyat üst sınırını girin (örn. 1000): "))
    return preferences

def encode_preferences(preferences, label_encoders):
    encoded_preferences = {}
    for key, value in preferences.items():
        if key in label_encoders:
            le = label_encoders[key]
            if value not in le.classes_:
                closest_value = min(le.classes_, key=lambda x: abs(len(x) - len(value)))
                encoded_preferences[key] = le.transform([closest_value])[0]
            else:
                encoded_preferences[key] = le.transform([value])[0]
        else:
            encoded_preferences[key] = value
    return encoded_preferences

def decode_predictions(predictions, label_encoders):
    decoded_predictions = {}
    for key, value in predictions.items():
        if key in label_encoders:
            le = label_encoders[key]
            decoded_predictions[key] = le.inverse_transform([int(value)])[0]
        else:
            decoded_predictions[key] = value
    return decoded_predictions

def recommend_laptops(model, scaler, label_encoders, df, X_scaled, num_recommendations=5):
    print("Merhaba! Size uygun bir bilgisayar bulmanıza yardımcı olacağım.")
    preferences = get_user_preferences()
    encoded_preferences = encode_preferences(preferences, label_encoders)

    max_price = preferences['Price']
    allowed_price = max_price * 1.25

    input_data = np.array([list(encoded_preferences.values())])
    input_data_scaled = scaler.transform(input_data)
    input_data_lstm = input_data_scaled.reshape((input_data_scaled.shape[0], input_data_scaled.shape[1], 1))

    predictions = model.predict(input_data_lstm)
    predictions_inversed = scaler.inverse_transform(predictions)
    predicted_values = predictions_inversed[0]

    # Cosine similarity hesaplayarak en benzer laptopları bulalım
    similarities = cosine_similarity([predicted_values], X_scaled)[0]
    most_similar_indices = similarities.argsort()[::-1]

    print("Önerilen bilgisayarlar:")
    recommended_count = 0
    for index in most_similar_indices:
        if recommended_count >= num_recommendations:
            break
        recommended_laptop = df.iloc[index]
        if recommended_laptop['Price'] <= allowed_price:
            decoded_recommendation = decode_predictions(recommended_laptop.to_dict(), label_encoders)
            print(f"Laptop ID: {index + 1}")
            for key, value in decoded_recommendation.items():
                print(f"{key}: {value}")
            print("\n")
            recommended_count += 1

# Veriyi yeniden ölçeklendirelim, yalnızca özellik sütunlarıyla
X_scaled = scaler.transform(X)

# Önerileri başlatalım
recommend_laptops(model, scaler, label_encoders, df, X_scaled)


Epoch 1/50
33/33 [==============================] - 3s 23ms/step - loss: 440763456.0000 - val_loss: 496092032.0000
Epoch 2/50
33/33 [==============================] - 0s 7ms/step - loss: 440700896.0000 - val_loss: 496007968.0000
Epoch 3/50
33/33 [==============================] - 0s 7ms/step - loss: 440651744.0000 - val_loss: 495976160.0000
Epoch 4/50
33/33 [==============================] - 0s 7ms/step - loss: 440626368.0000 - val_loss: 495950944.0000
Epoch 5/50
33/33 [==============================] - 0s 7ms/step - loss: 440604416.0000 - val_loss: 495927776.0000
Epoch 6/50
33/33 [==============================] - 0s 7ms/step - loss: 440583904.0000 - val_loss: 495905696.0000
Epoch 7/50
33/33 [==============================] - 0s 7ms/step - loss: 440564000.0000 - val_loss: 495884320.0000
Epoch 8/50
33/33 [==============================] - 0s 8ms/step - loss: 440544672.0000 - val_loss: 495863584.0000
Epoch 9/50
33/33 [==============================] - 0s 7ms/step - loss: 440525696.0000 